In [57]:
import os
import nltk
import json
import re
import pandas as pd
import numpy as np

In [3]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [6]:
len(nltk.corpus.stopwords.words('english'))

179

In [78]:
features=['title', 'id', 'release_date', 'lyrics']

In [79]:
df=pd.DataFrame()
for dirname, _, filenames in os.walk('data\json files'):
    for filename in filenames:
        with open(os.path.join(dirname, filename)) as f:
            aux = json.load(f)
            df_tmp=pd.DataFrame()
            Artist= np.full((len(aux['songs'])), aux['name'])
            df_tmp['artist']=Artist
            for f in features:
                col=[song[f] for song in aux['songs']]
                df_tmp[f]=col
            df=pd.concat([df,df_tmp])

In [115]:
tmp=[]
tags=['remix','Remix','live','Live', ' Version', ' ver', 'version', 'Demo', 'demo' , 'Reprise', 'Copy', 'Mix']

for title in df['title']:
    for tag in tags:
        r=title.find(tag)
        if r != -1:
            tmp.append(title)

In [129]:
len(tmp)

1476

In [127]:
for title in tmp:
    df= df[df.title != title]

In [145]:
for f in features:
    print("number null value of {}: ".format(f),sum(df[f].isnull()))


number null value of title:  0
number null value of id:  0
number null value of release_date:  1408
number null value of lyrics:  0


In [146]:
df.dropna(subset = ["lyrics"], inplace=True)